In [2]:
import sys
import os
import dtale
import dtale.app as dtale_app
dtale_app.JUPYTER_SERVER_PROXY = True
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config
config_info = {
  "canalyst_api_key": "",
  "s3_access_key_id": "",
  "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}
config = Config(config=config_info)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

A configuration file has been created for you in 
/home/canalyst_candas/canalyst/keys.json. For Excel file downloads and scenario 
mapping, you'll need to add an API key to the 'canalyst_api_key' 
value. Visit https://app.canalyst.com/settings/api-tokens to create 
and retrieve your API key. You will also need to retrieve your 
S3 Access ID and Secret Key and fill in 'canalyst_s3_id' and 
'canalyst_s3_key' with those values, respectively. If using a 
Jupyter Notebook, stop and restart the notebook for the changes to 
take effect. If using a Python/iPython session, quit the current 
session and start a new one.


## JPM Read Through for Fixed Income Trading with GS

### First we search for common time series between JPM and GS with 'FICC Fixed Income Markets'

In [27]:
canalyst_search = cd.Search(config)
df = canalyst_search.search_time_series(ticker=['JPM US'],
                                        unit_type='currency',
                     time_series_description='asset management equity AUM')[['ticker','time_series_description','time_series_name','search_matches']]
df= df[df['search_matches']>1]

Building search index...
No.of.unique tickers:  1
No.of.unique time series:  77


In [28]:
df
# gs z_JAJJLJ0162_MO_OS_Equityinvestments_1
# jpm z_3FRM7X0148_MO_OS_EquityNetAssetFlow

,ticker,time_series_description,time_series_name,search_matches
50378,JPM US,"Asset & Wealth Management Asset management, administration and commissions, mm",z_3FRM7X0148_MO_OS_AssetWealthManagementAssetmanagementadministrationandcommissions,4
50189,JPM US,"Total Asset Management fees, mm",z_3FRM7X0148_MO_OS_totalassetmanagementfees,2
50205,JPM US,"Total Asset Management fees, mm",z_3FRM7X0148_MO_OS_totalassetmanagementfees_1,2
50210,JPM US,"Multi-assets & Alternatives Net Asset Flow, bn",z_3FRM7X0148_MO_OS_multiassetsalternativesnetassetflow,2
50209,JPM US,"Equity Net Asset Flow, bn",z_3FRM7X0148_MO_OS_EquityNetAssetFlow,2
50320,JPM US,"Corporate & Investment Bank Asset management, administration and commissions, mm",z_3FRM7X0148_MO_OS_CorporateInvestmentBankAssetmanagementadministrationandcommissions,2
50260,JPM US,"Consumer & Community Banking Asset management, administration and commissions, mm",z_3FRM7X0148_MO_OS_ConsumerCommunityBankingAssetmanagementadministrationandcommissions,2
50354,JPM US,"Commercial Banking Asset management, administration and commissions, mm",z_3FRM7X0148_MO_OS_CommercialBankingAssetmanagementadministrationandcommissions,2
50411,JPM US,"Asset management, administration and commissions",z_3FRM7X0148_MO_IS_assetmanagement,2
50382,JPM US,"Asset & Wealth Management Total Revenue, mm",z_3FRM7X0148_MO_OS_AssetWealthManagementTotalRevenue,2


### Now that we have our series, let's pull the data from Canalyst

In [29]:
ticker = "JPM US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)

df_jpm_equity_flows = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      is_historical=True,
                      time_series_name='z_3FRM7X0148_MO_OS_EquityNetAssetFlow')[['ticker','time_series_name','period_end_date','value']]

### Next we get GS data from Canalyst

In [33]:
ticker = "GS US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)
df_gs_equity_investments = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_JAJJLJ0162_MO_OS_Equityinvestments_1')[['ticker','time_series_name','period_end_date','value','is_driver']]

### Now we use the regress_dataframe function from Candas DataReader

#### JPM ficc trading vs GS ficc trading

In [37]:
df = pd.merge(df_jpm_equity_flows,df_gs_equity_investments,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

Dep. Variable:,value_y,R-squared:,0.493
Model:,OLS,Adj. R-squared:,0.443
Method:,Least Squares,F-statistic:,9.737
Date:,"Mon, 10 Jan 2022",Prob (F-statistic):,0.0109
Time:,19:52:32,Log-Likelihood:,-261.78
No. Observations:,12,AIC:,527.6
Df Residuals:,10,BIC:,528.5
Df Model:,1,,
Covariance Type:,nonrobust,,


In [35]:
df.columns=['period_end_date','JPM US','GS US']
dtale.show(df.dropna())